# Lambda Treasure Hunt - CS BW2

https://github.com/gyhou/CS-Build-Week-2
---
**Overview**

You start the adventure unworthy, unequipped and anonymous. Your first task is to traverse the island and build a map for your personal use. Along the way, you will discover equipment, treasure and clues which will assist you on your quest for power, wealth and glory.

To efficiently traverse an island maze, collect treasure, solve puzzles, unearth powerful artifacts and more.

Glory and riches await the victors!

# Build a map

## Empty Map

In [0]:
# graph = {}

## Complete Road Map

- 500 rooms
- Shop: 1 (Sell treasures for gold)
- Pirate Rye: 467 (Exchange 1000g to change name)

### After changing name
- Wishing Well: 55 (Get clue for mining location)
- Shrines: 22, 374, 461, 491 (Unlock abilities)

In [0]:
# Upload graph file
from treasure_map import graph_map
graph = graph_map

# Initialization

In [0]:
import json
import time
import random
import requests
# Upload token
with open('george_token', 'r') as auth_token:
  token = auth_token.read()

In [3]:
def init(token):
  url = 'https://lambda-treasure-hunt.herokuapp.com/api/adv/init/'
  result = requests.get(url, 
                         headers={'Authorization': f'Token {token}'}).json()
  if len(result['errors']) > 0:
    print(result['errors'])                       
  if result['room_id'] not in graph:
    new_room = {}
    for direction in result['exits']:
        new_room[direction] = "?"
    graph[result['room_id']] = [new_room, result]
  return result
current = init(token)
current

{'cooldown': 1.0,
 'coordinates': '(58,61)',
 'description': "You are standing besides a large well. A sign next the well reads 'EXAMINE WELL, FIND WEALTH'.",
 'elevation': 0,
 'errors': [],
 'exits': ['n'],
 'items': [],
 'messages': [],
 'players': ['Labyrinth65',
  'Aaron The Beast',
  'Michael Mangus',
  'Kevin Smith',
  'Chance',
  'DORA_BELME',
  'Hartman',
  'Ad-R0ck'],
 'room_id': 555,
 'terrain': 'NORMAL',
 'title': 'Wishing Well'}

# Status, Inventory

In [4]:
def status_check():
  url = 'https://lambda-treasure-hunt.herokuapp.com/api/adv/status/'
  result = requests.post(url, 
                         headers={'Content-Type':'application/json',
                                  'Authorization': f'Token {token}'}).json()
  if len(result['errors']) > 0:
    print(result['errors'])   
  return result
status_check()

{'abilities': ['pray', 'mine', 'fly', 'dash', 'warp', 'recall'],
 'bodywear': 'well-crafted jacket',
 'cooldown': 1.0,
 'encumbrance': 4,
 'errors': [],
 'footwear': 'well-crafted boots',
 'gold': 12400,
 'has_mined': True,
 'inventory': ['poor jacket', 'nice jacket'],
 'messages': [],
 'name': '[George H]',
 'snitches': 70,
 'speed': 104,
 'status': [],
 'strength': 20}

## Equipment

- Bodywear: Increase strength
- Footwear: Increase speed

In [0]:
def equip_item(item_name):
  url = 'https://lambda-treasure-hunt.herokuapp.com/api/adv/wear/'
  data = f'{{"name":"{item_name}"}}'
  result = requests.post(url, data=data,
                         headers={'Content-Type':'application/json',
                                  'Authorization': f'Token {token}'}).json()
  if len(result['errors']) > 0:
    print(result['errors'])                  
  print(result)
  return result     
# equip_item('well-crafted jacket') # 21
equip_item('well-crafted boots') # speed 104

{'name': '[George H]', 'cooldown': 1.0, 'encumbrance': 6, 'strength': 20, 'speed': 104, 'gold': 12400, 'bodywear': 'well-crafted jacket', 'footwear': 'well-crafted boots', 'inventory': ['well-crafted boots', 'amazing treasure'], 'abilities': ['pray', 'mine', 'fly', 'dash', 'warp', 'recall'], 'status': [], 'has_mined': True, 'errors': [], 'messages': ['You wear well-crafted boots'], 'snitches': 59}


{'abilities': ['pray', 'mine', 'fly', 'dash', 'warp', 'recall'],
 'bodywear': 'well-crafted jacket',
 'cooldown': 1.0,
 'encumbrance': 6,
 'errors': [],
 'footwear': 'well-crafted boots',
 'gold': 12400,
 'has_mined': True,
 'inventory': ['well-crafted boots', 'amazing treasure'],
 'messages': ['You wear well-crafted boots'],
 'name': '[George H]',
 'snitches': 59,
 'speed': 104,
 'status': [],
 'strength': 20}

# Actions

- Movement (move, fly, dash, recall, warp)
- Treasure (take/sell)
- Pray (gain abilitie)
- Mine (Coin)

## Movement

- Move, Fly, Dash, Recall, Warp

### Breathe First Search

- Finds the shortest path to the destination

#### Qeue Class

In [0]:
# Helper class
class Queue():
  def __init__(self):
      self.queue = []
  def enqueue(self, value):
      self.queue.append(value)
  def dequeue(self):
      if self.size() > 0:
          return self.queue.pop(0)
      else:
          return None
  def size(self):
      return len(self.queue)

#### BFS

In [0]:
def bfs(starting_room, destination):
  queue = Queue()
  queue.enqueue([starting_room])
  visited = set()
  while queue.size() > 0:
    path = queue.dequeue()
    room_id = path[-1]
    if room_id not in visited:
      if room_id == destination:
        return path
      else:
        for direction in graph[room_id][0]:
          # if graph[room_id][0][direction] != "?":
            new_path = path.copy()
            new_path.append(graph[room_id][0][direction])
            queue.enqueue(new_path)
      visited.add(room_id)
  return []

In [0]:
# Test
bfs(current['room_id'], 555), bfs(500, 555)

([], [500, 504, 506, 507, 511, 516, 528, 555])

### Move/Fly

- One room traversal
- Base 15s CD

#### Basic movement

Move - no buff or penalty

**Wise Explorer Buff**

- Predict the destination room id to reduce CD by 50%


---


After obtaining **Fly** ability

Buffs
- Instant CD when going downhill
- -10% CD in normal rooms

Penalty
- +10s CD in CAVE terrains

In [0]:
# Movement
def movement(current_room_id, direction, next_room_id=None):
  if graph[current_room_id][1]['terrain'] == 'CAVE':
    url = 'https://lambda-treasure-hunt.herokuapp.com/api/adv/move/'
  else:
    # Once you acquire flying ability
    url = 'https://lambda-treasure-hunt.herokuapp.com/api/adv/fly/'
  if next_room_id is not None and next_room_id != "?":
    data = f'{{"direction":"{direction}","next_room_id": "{next_room_id}"}}'
    if graph[next_room_id][1]['terrain'] == 'CAVE':
      url = 'https://lambda-treasure-hunt.herokuapp.com/api/adv/move/'
  else:
    data = f'{{"direction":"{direction}"}}'
  
  result = requests.post(url, data=data,
                         headers={'Content-Type':'application/json',
                                  'Authorization': f'Token {token}'}).json()
  if len(result['errors']) > 0:
    print(result['errors'])
    return
  # Update room_id in graph
  inverse_directions = {"n": "s", "s": "n", "e": "w", "w": "e"}
  if result['room_id'] not in graph:
    graph[result['room_id']] = {}
    for road in result['exits']:
      graph[result['room_id']][road] = "?"
    new_room = {}
    for road in result['exits']:
      new_room[road] = "?"
    graph[result['room_id']] = [new_room, result]
  graph[current_room_id][0][direction] = result['room_id']
  graph[result['room_id']][0][inverse_directions[direction]] = current_room_id

  return result

In [0]:
current = init(token)
time.sleep(current['cooldown'])
# Run forever until interrupted
prev_move = None
inverse_directions = {"n": "s", "s": "n", "e": "w", "w": "e"}
while True:
  directions = current['exits']
  if len(directions) > 1 and prev_move:
    directions.remove(inverse_directions[prev_move])
  direction = random.choice(directions)
  print(current['room_id'], direction, graph[current['room_id']][0])
  next_room = movement(current['room_id'], direction, graph[current['room_id']][0][direction])
  print(current['room_id'], next_room['room_id'], next_room['messages'])
  time.sleep(next_room['cooldown'])

  quality = ['sparkling','brilliant','dazzling','spectacular','amazing','golden','well-crafted','exquisite']
  if len(next_room['items']) > 0:
    print(f'{next_room["items"]} found in Room[{next_room["room_id"]}]')
    for item in next_room['items']:          
      item_grade = item.split(' ')[0]
      if item_grade in quality:
        time.sleep(next_room['cooldown'])
        next_room = take_item(item)
  prev_move = direction
  current = next_room
  time.sleep(next_room['cooldown'])

In [0]:
# 815 778
bfs(current['room_id'], 779)

[648, 610, 779]

### Dash

After obtaining **Dash** ability
- Cover many rooms in one direction quickly
- 15s CD + 0.5s CD for each room dashed

In [0]:
def dash(direction, num_rooms, next_room_ids):
  url = 'https://lambda-treasure-hunt.herokuapp.com/api/adv/dash/'
  next_room_ids = [str(room_id) for room_id in next_room_ids]
  next_room_ids = ','.join(next_room_ids)
  data = f'{{"direction":"{direction}","num_rooms":"{num_rooms}","next_room_ids":"{next_room_ids}"}}'
  result = requests.post(url, data=data,
                         headers={'Content-Type':'application/json',
                                  'Authorization': f'Token {token}'}).json()
  if len(result['errors']) > 0:
    print(result['errors'])                  
  # print(result)
  return result                                 

### Recall

**Room 492**
- Recall to your starting location Room_id : 0
- Base 15s CD

In [0]:
def recall():
  url = 'https://lambda-treasure-hunt.herokuapp.com/api/adv/recall/'
  result = requests.post(url, headers={'Content-Type':'application/json',
                                       'Authorization': f'Token {token}'})
  return result.json()
# current = recall()

### Warp

**Room 374**

- Warp to an alternate dimension where you can compete with other players to search for the mysterious golden snitch.
- Note that warp travel is dangerous so you must be have both bodywear and footwear equipped to warp.

In [0]:
def warp():
  url = 'https://lambda-treasure-hunt.herokuapp.com/api/adv/warp/'
  result = requests.post(url, 
                         headers={'Content-Type':'application/json',
                                  'Authorization': f'Token {token}'}).json()
  # print(result)
  time.sleep(result['cooldown'])
  return result
# current = warp()
# current

## Take Treasure

- Pick up treasures to sell at shop to gain gold
- Player can hold up to player's [strength stats] items
- When a player's inventory is full +100% CD
- Tiny, Small, Shiny, Great, Amazing, Spectacular, Dazzling, Brilliant, Sparkling
- Use for transmogrify

In [0]:
# Take treasure
def take_item(treasure):
  url = 'https://lambda-treasure-hunt.herokuapp.com/api/adv/take/'
  data = f'{{"name":"{treasure}"}}'
  result = requests.post(url, data=data,
                         headers={'Content-Type':'application/json',
                                  'Authorization': f'Token {token}'}).json()
  if len(result['errors']) > 0:
    print(result['errors'])
  else:
    print(f'Taking {treasure}!')
  return result
# take_item('golden snitch')
# take_item('nice jacket')

In [0]:
# Sell Treasure
def sell_item(treasure):
  url = 'https://lambda-treasure-hunt.herokuapp.com/api/adv/sell/'
  data = f'{{"name":"{treasure}", "confirm":"yes"}}'
  result = requests.post(url, data=data,
                         headers={'Content-Type':'application/json',
                                  'Authorization': f'Token {token}'}).json()
  if len(result['errors']) > 0:
    print(result['errors'])
  print(f'{result["messages"]}')
  return result

## Selling items

In [0]:
current = init(token)
time.sleep(current['cooldown'])
current = map_traversal(current, 1, True)
# time.sleep(current['cooldown'])
items = status_check()['inventory']
time.sleep(1)
low_quality = ['small','poor','terrible','nice','shiny']
for item in items:
  item_quality = item.split(' ')[0]
  if item_quality in low_quality:
    current = sell_item(item)
    time.sleep(current['cooldown'])

Recall path (2) < base path (22)
RECALLING to Room[0]! CD: 6.0s
Heading to the Room[1].. [0, 1]
Room[0] to Room[1]
CD: 2.70, ['You have flown west.', 'Flight Bonus: -10% CD', 'Wise Explorer: -50% CD']
["Thanks, I'll take that poor jacket.", 'You have received 200 gold.']
["Thanks, I'll take that nice jacket.", 'You have received 500 gold.']
["Thanks, I'll take that poor boots.", 'You have received 200 gold.']
["Thanks, I'll take that terrible jacket.", 'You have received 100 gold.']
["Thanks, I'll take that poor boots.", 'You have received 200 gold.']
["Thanks, I'll take that poor boots.", 'You have received 200 gold.']
["Thanks, I'll take that terrible jacket.", 'You have received 100 gold.']
["Thanks, I'll take that poor boots.", 'You have received 200 gold.']


## Name Change

Pirate Rye (467) - Use 1000g to change name to unlock abilities and pray

In [0]:
def name_change(new_name):
  url = 'https://lambda-treasure-hunt.herokuapp.com/api/adv/change_name/'
  data = f'{{"name":"{new_name}", "confirm":"aye"}}'
  result = requests.post(url, data=data,
                         headers={'Content-Type':'application/json',
                                  'Authorization': f'Token {token}'}).json()
  print(f'Changing name to {new_name}!')
  return result
# name_change('Jon Doe')

## Pray at Shrine

Need to change name first to pray and unlock abilities
- 22 - Fly: Good for all terrains except cave.
- 374 - Warp: Must be have both bodywear and footwear equipped to warp.
- 461 - Dash: Cover many rooms in one direction quickly
- 492 - Recall: Recall to starting Room[0]



In [0]:
def pray():
  url = 'https://lambda-treasure-hunt.herokuapp.com/api/adv/pray/'
  result = requests.post(url, headers={'Content-Type':'application/json',
                                       'Authorization': f'Token {token}'})
  print(result)
  return result.json()
# current = pray()
# time.sleep(current['cooldown'])

## Transmogrify

Room 495

- Trade One item and one Lambdacoin for an bodywear or footwear item

**Different grades of items**
- Appalling, Terrible, Poor, Nice, Well-crafted, Exquisite

In [0]:
def transmogrify(item_name):
  url = 'https://lambda-treasure-hunt.herokuapp.com/api/adv/transmogrify/'
  data = f'{{"name":"{item_name}"}}'
  result = requests.post(url, data=data,
                         headers={'Content-Type':'application/json',
                                  'Authorization': f'Token {token}'}).json()
  if len(result['errors']) > 0:
    print(result['errors'])
  print(result['messages'])                                
  return result
# transmogrify('shiny treasure')
# transmogrify('nice boots')

In [0]:
# time.sleep(current['cooldown'])
items = status_check()['inventory']
time.sleep(1)
quality = ['nice','well-crafted','shiny','great']
for item in items:
  # item_quality = item.split(' ')[0]
  # if item_quality in quality:
    current = transmogrify(item)
    time.sleep(current['cooldown'])

['Your poor jacket transmogrified into poor jacket!']


# BFS Map Traversal

In [0]:
def map_traversal(current_room, destination, pick_item=False, alt_dim=False):
  if destination > 499:
    alt_dim = True
  # Recall or walk?
  recall_path = bfs(0, destination)
  base_path = bfs(current_room['room_id'], destination)
  warp_recall_path = bfs(500, destination)

  if alt_dim:
    if len(warp_recall_path) + 7 < len(base_path):
      print(f'Recall path ({len(warp_recall_path)}) < base path ({len(base_path)})')
      current_room = recall()
      print(f'RECALLING to Room[0]! CD: {current_room["cooldown"]}')
      time.sleep(current_room['cooldown'])
      current_room = warp()
      print(f'Warping to Room[500]! CD: {current_room["cooldown"]}')
      time.sleep(current_room['cooldown'])
      base_path = warp_recall_path
  elif len(recall_path) + 3 < len(base_path):
    print(f'Recall path ({len(recall_path)}) < base path ({len(base_path)})')
    current_room = recall()
    print(f'RECALLING to Room[0]! CD: {current_room["cooldown"]}s')
    base_path = recall_path
    time.sleep(current_room['cooldown'])
  # Base path = list of room_ids to traverse to destination
  print(f'Heading to the Room[{destination}].. {base_path}')
  while len(base_path) > 1:
    path_exits = graph[current_room['room_id']][0]
    for direction, room_id in path_exits.items():
      if room_id == base_path[1]:
        # Figure out the direction of next room
        next_move = direction
        break
    # How many rooms available to Dash?
    next_room_ids = [base_path[1]]
    if len(base_path) > 2:
      for index in range(1,len(base_path)-1):
        # If direction the same, add to list
        if next_move in graph[base_path[index]][0]:
          if graph[base_path[index]][0][next_move] == base_path[index+1]:
            next_room_ids.append(base_path[index+1])
          # If not, stop the loop
          else:
            break
        else:
          break
    if '?' in next_room_ids:
      next_room_ids.remove('?')
    # Check length of eligible rooms to Dash
    if len(next_room_ids) > 2:
      print(f'Dash - Room[{current_room["room_id"]}] {next_move} to {next_room_ids}')
      end_room = dash(next_move, len(next_room_ids), next_room_ids)
      base_path = base_path[len(next_room_ids):]
    # If not, move normally
    else:
      # print(current_room['room_id'], next_move, path_exits[next_move])
      end_room = movement(current_room['room_id'], next_move, path_exits[next_move])
      print(f'Room[{current_room["room_id"]}] to Room[{end_room["room_id"]}]')
      # print(end_room)
      base_path.pop(0)
    print(f"CD: {end_room['cooldown']:.2f}, {end_room['messages']}")
    
    
    quality = ['sparkling','brilliant','dazzling','spectacular','amazing','golden','well-crafted','exquisite']
    if pick_item:
      if len(end_room['items']) > 0:
        print(f'{end_room["items"]} found in Room[{end_room["room_id"]}]')
        for item in end_room['items']:          
          item_grade = item.split(' ')[0]
          if item_grade in quality:
            time.sleep(end_room['cooldown'])
            end_room = take_item(item)
            if item == 'golden snitch':
              return end_room

    current_room = end_room
    time.sleep(current_room['cooldown'])
  return current_room

## BFS Destination

In [23]:
# import random
current = init(token)
time.sleep(current['cooldown'])
shop = 1
wishing_well_id = 55
pirate_rye_id = 467
transmogriphier_room = 495
# while True:
# room_num = random.randint(500,999)
current = map_traversal(current, 999, True)

Heading to the Room[999].. [980, 999]
Room[980] to Room[999]
CD: 2.70, ['You have flown south.', 'Flight Bonus: -10% CD', 'Wise Explorer: -50% CD']


# Lambda Coin

## Examine Well for clue

- Room 55
- Decipher clue to get mining room location

In [0]:
def wishing_well():
  url = 'https://lambda-treasure-hunt.herokuapp.com/api/adv/examine/'
  data = f'{{"name":"Well"}}'
  result = requests.post(url, data=data,
                         headers={'Content-Type':'application/json',
                                  'Authorization': f'Token {token}'}).json()
  if len(result['errors']) > 0:
    print(result['errors'])                              
  # print(result['description'])
  return result
# message = wishing_well()
# message

## Decipher Message

In [0]:
# import ls8 CPU
from ls8_cpu import *
def msg_decode(message):
  cpu = CPU()
  split_msg = message.split('.')[3]
  cpu.load(split_msg)
  decode_msg = cpu.run()
  join_msg = "".join(decode_msg)
  mining_room_id = int(''.join(i for i in join_msg if i.isdigit()))
  print(join_msg)
  return mining_room_id
# msg_decode(mining_code['description'])

## Blockchain Proof

In [0]:
def get_last_proof():
  url = 'https://lambda-treasure-hunt.herokuapp.com/api/bc/last_proof/'
  result = requests.get(url, 
                        headers={'Content-Type':'application/json',
                                 'Authorization': f'Token {token}'}).json()
  return result
# get_last_proof()

In [0]:
def proof_of_work(last_proof, current_level):
    """
    Get the last valid proof to use to mine a new block. 
    Also returns the current difficulty level, which is the number of 0's 
    required at the beginning of the hash for a new proof to be valid.
    The proof of work algorithm for this blockchain is 
    not the same as we used in class.
    """
    start = timer()
    print(f"\nLast proof: {last_proof} -- Searching for next proof..\n")
    proof = 99999999
    while valid_proof(last_proof, proof, current_level) is False:
      proof -= 1
    print(f"Proof found: {proof} in {timer() - start:.3f}s")
    return proof

In [0]:
import hashlib
def valid_proof(last_proof, proof, current_level):
    """
    Does hash(last_proof, proof) contain N leading zeroes, 
    where N is the current difficulty level?
    Use sha256 in hashlib
    """
    guess = f"{last_proof}{proof}".encode()
    guess_hash = hashlib.sha256(guess).hexdigest()

    return guess_hash[:current_level] == "0"*current_level

In [0]:
def mine_coin(new_proof):
  url = 'https://lambda-treasure-hunt.herokuapp.com/api/bc/mine/'
  data = f'{{"proof":{new_proof}}}'
  result = requests.post(url, data=data,
                         headers={'Content-Type':'application/json',
                                  'Authorization': f'Token {token}'}).json()
  return result

## Mining Coin

In [0]:
from timeit import default_timer as timer
# current = init(token)
# time.sleep(current['cooldown'])
def mining_coin():
  while True:
    # Get the last proof from the server
    current_proof = get_last_proof()
    time.sleep(1)
    proof = current_proof['proof']
    current_level = current_proof['difficulty']

    new_proof = proof_of_work(proof, current_level)
    reponse = mine_coin(new_proof)
    print(reponse)
    time.sleep(reponse['cooldown'])
    if len(reponse['errors']) == 0:
      if reponse['messages'] == ['New Block Forged']:
        break
    else:
      print(reponse['errors'])
  return reponse
# mining_coin()

## Check Coin Balance

In [15]:
def coin_balance():
  url = 'https://lambda-treasure-hunt.herokuapp.com/api/bc/get_balance/'
  result = requests.get(url, 
                        headers={'Content-Type':'application/json',
                                 'Authorization': f'Token {token}'}).json()
  time.sleep(1)
  return result
coin_balance()['messages']

['You have a balance of 0.0 Lambda Coins']

# Automate Mining

In [0]:
from timeit import default_timer as timer
current = init(token)
time.sleep(current['cooldown'])
# Run forever until interrupted
while True:
  # Head to wishing well 55
  current_room = map_traversal(current, 55, True)
  time.sleep(current_room['cooldown'])
  # get message
  mining_code = wishing_well()
  time.sleep(mining_code['cooldown'])
  # decode message
  mine_room_id = msg_decode(mining_code['description'])
  # head to decoded location
  current_room = map_traversal(current_room, mine_room_id, True)
  current = graph[mine_room_id][1]
  time.sleep(current_room['cooldown'])
  # mine
  mining_coin()
  print(coin_balance()['messages'])

# Find Golden Snitch

In [22]:
from timeit import default_timer as timer
current = init(token)
time.sleep(current['cooldown'])
wait_new_message = True
# Run forever until interrupted
while True:
  # Head to large wishing well 555
  current = map_traversal(current, 555, True, True)
  time.sleep(current['cooldown'])
  # get message
  snitch_code = wishing_well()
  time.sleep(snitch_code['cooldown'])
  # decode current message
  old_snitch_id = msg_decode(snitch_code['description'])
  print(f'Current golden snitch location: {old_snitch_id}')
  start = timer()
  while wait_new_message:
    new_msg = wishing_well()
    # print(f'Waiting for new clue for golden snitch in {new_msg["cooldown"]}s CD')
    time.sleep(new_msg['cooldown'])
    # decode new message
    new_snitch_id = msg_decode(new_msg['description'])
    if timer() - start > 15:
      print(f'Waited for {timer() - start:.2f}s..')
    if old_snitch_id != new_snitch_id or (timer() - start > 30):
      wait_new_message = False
    old_snitch_id = new_snitch_id
  # head to decoded location
  current = map_traversal(current, old_snitch_id, True, True)
  wait_new_message = True
  time.sleep(current['cooldown'])

Heading to the Room[555].. [500, 504, 506, 507, 511, 516, 528, 555]
Dash - Room[500] n to [504, 506, 507]
CD: 7.50, ['You have dashed to room 504.', 'You have dashed to room 506.', 'You have dashed to room 507.']
['shiny treasure', 'shiny treasure', 'nice jacket', 'poor boots'] found in Room[507]
Room[507] to Room[511]
CD: 2.70, ['You have flown west.', 'Flight Bonus: -10% CD', 'Wise Explorer: -50% CD']
Room[511] to Room[516]
CD: 2.70, ['You have flown west.', 'Flight Bonus: -10% CD', 'Wise Explorer: -50% CD']
Room[516] to Room[528]
CD: 2.70, ['You have flown south.', 'Flight Bonus: -10% CD', 'Wise Explorer: -50% CD']
['nice jacket', 'nice boots'] found in Room[528]
Room[528] to Room[555]
CD: 2.70, ['You have flown south.', 'Flight Bonus: -10% CD', 'Wise Explorer: -50% CD']
Find the snitch in room 869
Current golden snitch location: 869
Waiting for new clue for golden snitch in 1.0s CD
Find the snitch in room 869
Waiting for new clue for golden snitch in 1.0s CD
Find the snitch in room

KeyboardInterrupt: ignored

# Traverse Unknown Map

In [0]:
# Mapping out alternate dimension (room_id > 499)
current = init(token)
time.sleep(current['cooldown'])
while True:
  current_exits = graph[current['room_id']][0]
  directions = []
  for direction, room_id in current_exits.items():
    # If adjacent room_id not visited yet, add that direction
    if room_id == "?":
      directions.append(direction)
  if len(directions) == 0:
    print('All adjacent rooms visited..')
    end_room = map_traversal(current, "?", True, True)
  else:
    next_move = random.choice(directions)
    print(f"Room[{current['room_id']}] to the unknown.. ({len(graph)}/1000)")
    end_room = movement(current['room_id'], next_move)
    print(f"CD: {end_room['cooldown']}, {end_room['messages']}")

  if len(end_room['errors']) > 0:
    print(f"Error: {end_room['errors']}")
    break

  quality = ['sparkling','brilliant','dazzling','spectacular','amazing','great','golden','nice','well-crafted','exquisite']
  if len(end_room['items']) > 0:
    print(f'{end_room["items"]} found in Room[{end_room["room_id"]}]')
    for item in end_room['items']:          
      item_grade = item.split(' ')[0]
      if item_grade in quality:
        time.sleep(end_room['cooldown'])
        end_room = take_item(item)

  # print(end_room['room_id'], graph[end_room['room_id']][0])
  print(end_room)
  current = end_room
  time.sleep(current['cooldown'])

Room[747] to the unknown.. (986/1000)
CD: 9.0, ['You have flown north.', 'Flight Bonus: -10% CD']
{'room_id': 839, 'title': 'Darkness', 'description': 'You are standing on grass and surrounded by darkness.', 'coordinates': '(55,70)', 'elevation': 0, 'terrain': 'NORMAL', 'players': [], 'items': [], 'exits': ['s', 'w'], 'cooldown': 9.0, 'errors': [], 'messages': ['You have flown north.', 'Flight Bonus: -10% CD']}
Room[839] to the unknown.. (987/1000)
CD: 9.0, ['You have flown west.', 'Flight Bonus: -10% CD']
{'room_id': 911, 'title': 'Darkness', 'description': 'You are standing on grass and surrounded by darkness.', 'coordinates': '(54,70)', 'elevation': 0, 'terrain': 'NORMAL', 'players': [], 'items': [], 'exits': ['e', 'w'], 'cooldown': 9.0, 'errors': [], 'messages': ['You have flown west.', 'Flight Bonus: -10% CD']}
Room[911] to the unknown.. (988/1000)
CD: 9.0, ['You have flown west.', 'Flight Bonus: -10% CD']
{'room_id': 921, 'title': 'Darkness', 'description': 'You are standing on g

KeyboardInterrupt: ignored

In [0]:
len(graph)

1000